# how to build an animated chart of the yield curve to form a view of state of the economy.

The yield curve is a plot of the yield of different key US treasuries against their maturity. The yield curve provides important insight into investors’ predictions about the economy. “Inverted” yield curves have been shown to reliably predict recessions.

Normally, longer-dated treasuries yield more than shorter-dated ones. This is because there is more risk in lending money for longer. When the yield curve inverts, longer-dated treasuries yield less than shorter-dated ones. This happens when the market expects the Fed to lower interest rates in the future in anticipation of a recession.

Investors use an inverted yield curve as a sign to reduce risk in their portfolios.

We can build an animated chart of the US yield curve to form a view of the macroeconomic state.

In [1]:
# Imports and set up

# Start with importing NumPy, Matplotlib, and the OpenBB SDK. You can use the OpenBB SDK to get yield curve data for the St. Louis Fed’s data portal (FRED)

%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from openbb_terminal.sdk import openbb

font = {
    "family": "normal",
    "weight": "normal",
    "size": 12
}

plt.rc('font', **font)

# Set up the bond maturities you want to use to build the yield curve. There are conventions about which maturities to use, but there is no “right” answer. The point is to get a good representation across the maturities.

maturities = ['3m', '6m', '1y', '2y', '3y', '5y', '7y', '10y', '30y']

data = openbb.economy.treasury(
    instruments=["nominal"],
    maturities=maturities,
    start_date="1985-01-01"
)
data.columns = maturities

data["inverted"] = data["30y"] < data["3m"]

data.head()

,3m,6m,1y,2y,3y,5y,7y,10y,30y,inverted
TIME_PERIOD,,,,,,,,,,
1985-01-01,8.02,8.45,9.02,9.93,10.43,10.93,11.27,11.38,11.45,False
1985-02-01,8.56,8.87,9.29,10.17,10.55,11.13,11.44,11.51,11.47,False
1985-03-01,8.83,9.45,9.86,10.71,11.05,11.52,11.82,11.86,11.81,False
1985-04-01,8.22,8.71,9.14,10.09,10.49,11.01,11.34,11.43,11.47,False
1985-05-01,7.73,8.07,8.46,9.39,9.75,10.34,10.72,10.85,11.05,False


In [4]:
# Build the animated chart

# With Matplotlib, you can animate a chart. Animated charts offer a great way to visualize how data changes through time. First, set up the plots. This boilerplate code sets up the figure, creates the tick ranges and labels, and sets the axis labels.

# Initialize figure
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot([], [])

# Set the range of ticks
ax.set_xlim(0, 7)
ax.set_ylim(0, 20)

# Set the tick locations
ax.set_xticks(range(8))
ax.set_yticks([2, 4, 6, 8, 10, 12, 14, 16, 18])

# Set the axis labels
ax.set_xticklabels(["1m","3m","6m","1y","5y","10y","20y","30y"])
ax.set_yticklabels([2, 4, 6, 8, 10, 12, 14, 16, 18])

# Foce the y-axis labels to the left
ax.yaxis.set_label_position("left")
ax.yaxis.tick_left()

# Add the axis lables
plt.ylabel("Yield (%)")
plt.xlabel("Time to maturty")
# Next, build the animation functions.

def init_func():
    line.set_data([], [])
    plt.title("U.S. Treasury Bond Yield Curve")
    return line

def animate(i):
    x = range(0, len(maturities))
    y = data[maturities].iloc[i]
    dt_ = data.index[i].strftime("%Y-%m-%d")

    if data.inverted.iloc[i]:
        line.set_color("r")
    else:
        line.set_color("y")

    line.set_data(x, y)

    plt.title(f"U.S. Treasury Bond Yield Curve ({dt_})")
    return line,

ani = animation.FuncAnimation(
    fig,
    animate,
    init_func=init_func,
    frames=len(data.index),
    interval=5,
    blit=True
)